[ 가변 길이의 텍스트 처리를 위한 모델 ]
- 임베딩층 : nn.EmbeddingBag
- 구현 
  * 입력 텍스트의 길이를 측정해서 길이만큼 추출 후 진행

In [1]:
# [1] 모듈 로딩 및 데이터 준비
import torch
import torch.nn as nn 
from torchtext.datasets import AG_NEWS

c:\Users\knudc\anaconda3\envs\NLP\lib\site-packages\torchtext\datasets\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\knudc\anaconda3\envs\NLP\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
# 데이터 경로 준비
SAVE_DIR = './Data/'

In [3]:
AG_NEWS(SAVE_DIR)

c:\Users\knudc\anaconda3\envs\NLP\lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


(ShardingFilterIterDataPipe, ShardingFilterIterDataPipe)

In [4]:
trainDS, testDS = AG_NEWS()

In [16]:
type(trainDS)

torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe

In [5]:
for label, news in trainDS:
    print(label, news)
    break

3 Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


In [17]:
# [2] 데이터 로더 생성
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# 토크나이저, 단어사전 관련 모듈
from torchtext.data.utils import get_tokenizer           # 토크나이저 인스턴스 추출
from torchtext.vocab import build_vocab_from_iterator    # 데이터셋에서 단어사전 생성 함수
from nltk.corpus import stopwords                        # 불용어 처리 데이터셋

In [18]:
# 단어 사전 생성
UNK, PAD = '<UNK>','<PAD>'
STOPWORDS = stopwords.words('english')

# 토커나이저 생성
tokenizer = get_tokenizer('basic_english')

In [21]:
### ===> 토큰 제너레이터 함수 : 데이터 추출하여 토큰화 
def yield_tokens(data_iter):
    for label, text in data_iter:
        # 라벨, 뉴스 --> 텍스트 토큰화
        tokens = tokenizer(news)
        
        tokens = [ token for token in tokens if token not in STOPWORDS ]
        yield tokens

In [24]:
### ===> 토큰화 및 단어/어휘 사전 생성
VOCAB = build_vocab_from_iterator(yield_tokens(trainDS),
                                  specials= [PAD, UNK])

### <PAD> 인덱스 0으로 설정
VOCAB.set_default_index(VOCAB[PAD])

In [28]:
# 텍스트 정수 인코딩
text_pipeline = lambda x : VOCAB(tokenizer(x))

# 레이블 정수 인코딩
label_pipeline = lambda x : int(x) - 1

In [26]:
# 단어 사전 생성
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
### ===> DataLoader에서 배치크기만큼 데이터셋 반환 함수 
# 함수 기능 : 배치크기 만큼 데이터셋 로딩해서 토큰 + 텐서화 진행 후 반환
def collate_batch(batch):
    # 배치크기 만큼의 라벨, 텍스트, 오프셋 값 저장 변수 
    label_list, text_list, offsets = [], [], [0]
    
    # 1개씩 뉴스기사, 라벨 추출 해서 저장 
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))
         
        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
         
        # 다음 뉴스 읽기 위한 위치값 정보
        offsets.append(processed_text.size(0))
    
    # 배치 크기 만큼의 라벨 리스트 텐서화 진행     
    label_list = torch.tensor(label_list, dtype=torch.float32)
    
    # 배치 크기 만큼의 길이 위치값 텐서화
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    
    # 배치 크기 만큼의 뉴스 기사 리스트 텐서화
    text_list = torch.cat(text_list)
    
    # # 문장의 길이 일치
    # text_list = pad_sequence(text_list, batch_first=True, padding_value=0)
    
    return label_list.to(DEVICE), text_list.to(DEVICE), offsets.to(DEVICE)

In [ ]:
# 데이터로더 생성 
## DataLoader 생성
BATCH_SIZE = 5

trainDL = DataLoader( trainDS,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    collate_fn=collate_batch)

testDL = DataLoader( testDS,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    collate_fn=collate_batch)

In [36]:
cnt =5
for label, news, offset in trainDL:
    print(f'\nlabels   : {label.shape} => {label}')
    print(f'news     : {news.shape}')
    print(f'offset   : {offset}')
    cnt -= 1
    if cnt==0: break


labels   : torch.Size([5]) => tensor([0., 2., 0., 3., 0.])
news     : torch.Size([185])
offset   : tensor([  0,  49,  80, 116, 152])

labels   : torch.Size([5]) => tensor([0., 0., 0., 1., 3.])
news     : torch.Size([213])
offset   : tensor([  0,  53,  85, 136, 161])

labels   : torch.Size([5]) => tensor([2., 2., 3., 0., 1.])
news     : torch.Size([197])
offset   : tensor([  0,  53,  98, 125, 147])

labels   : torch.Size([5]) => tensor([3., 2., 0., 3., 0.])
news     : torch.Size([269])
offset   : tensor([  0,  48, 101, 149, 189])

labels   : torch.Size([5]) => tensor([0., 3., 2., 2., 2.])
news     : torch.Size([252])
offset   : tensor([  0,  60, 113, 146, 183])


In [ ]:
# 모델 클래스 정의 및 설계 